## CannyEdgeDetection

In [19]:
import numpy as np
import skimage
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import os
import scipy.misc as sm
from scipy import ndimage
from scipy.ndimage.filters import convolve
from scipy import misc
from PIL import Image
import PIL.ImageOps
import cv2

def rgb2gray(rgb):
    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

def load_data(dir_name,stop=None):    
    '''
    Load images from the "faces_imgs" directory
    Images are in JPG and we convert it to gray scale images
    '''
    imgs = []
    pic_names = []
    
    for filename in os.listdir(dir_name): 
        if os.path.isfile(dir_name + '/' + filename):
            img = mpimg.imread(dir_name + '/' + filename)
            img = rgb2gray(img)
            imgs.append(img)
            pic_names.append(filename)
        if len(pic_names) == stop:
            break
    return imgs, pic_names


def visualize(imgs, format=None, gray=False):
    plt.figure(figsize=(20, 40))
    for i, img in enumerate(imgs):
        if img.shape[0] == 3:
            img = img.transpose(1,2,0)
        plt_idx = i+1
        plt.subplot(2, 2, plt_idx)
        plt.imshow(img, format)
    plt.show()

class cannyEdgeDetector:
    def __init__(self, imgs, sigma=1, kernel_size=5, weak_pixel=75, strong_pixel=255, lowthreshold=0.05, highthreshold=0.15):
        self.imgs = imgs
        self.imgs_final = []
        self.img_smoothed = None
        self.gradientMat = None
        self.thetaMat = None
        self.nonMaxImg = None
        self.thresholdImg = None
        self.weak_pixel = weak_pixel
        self.strong_pixel = strong_pixel
        self.sigma = sigma
        self.kernel_size = kernel_size
        self.lowThreshold = lowthreshold
        self.highThreshold = highthreshold
        return 
    
    def gaussian_kernel(self, size, sigma=1):
        size = int(size) // 2
        x, y = np.mgrid[-size:size+1, -size:size+1]
        normal = 1 / (2.0 * np.pi * sigma**2)
        g =  np.exp(-((x**2 + y**2) / (2.0*sigma**2))) * normal
        return g
    
    def sobel_filters(self, img):
        Kx = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], np.float32)
        Ky = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], np.float32)

        Ix = ndimage.filters.convolve(img, Kx)
        Iy = ndimage.filters.convolve(img, Ky)

        G = np.hypot(Ix, Iy)
        G = G / G.max() * 255
        theta = np.arctan2(Iy, Ix)
        return (G, theta)
    

    def non_max_suppression(self, img, D):
        M, N = img.shape
        Z = np.zeros((M,N), dtype=np.int32)
        angle = D * 180. / np.pi
        angle[angle < 0] += 180


        for i in range(1,M-1):
            for j in range(1,N-1):
                try:
                    q = 255
                    r = 255

                   #angle 0
                    if (0 <= angle[i,j] < 22.5) or (157.5 <= angle[i,j] <= 180):
                        q = img[i, j+1]
                        r = img[i, j-1]
                    #angle 45
                    elif (22.5 <= angle[i,j] < 67.5):
                        q = img[i+1, j-1]
                        r = img[i-1, j+1]
                    #angle 90
                    elif (67.5 <= angle[i,j] < 112.5):
                        q = img[i+1, j]
                        r = img[i-1, j]
                    #angle 135
                    elif (112.5 <= angle[i,j] < 157.5):
                        q = img[i-1, j-1]
                        r = img[i+1, j+1]

                    if (img[i,j] >= q) and (img[i,j] >= r):
                        Z[i,j] = img[i,j]
                    else:
                        Z[i,j] = 0


                except IndexError as e:
                    pass

        return Z

    def threshold(self, img):

        highThreshold = img.max() * self.highThreshold;
        lowThreshold = highThreshold * self.lowThreshold;

        M, N = img.shape
        res = np.zeros((M,N), dtype=np.int32)

        weak = np.int32(self.weak_pixel)
        strong = np.int32(self.strong_pixel)

        strong_i, strong_j = np.where(img >= highThreshold)
        zeros_i, zeros_j = np.where(img < lowThreshold)

        weak_i, weak_j = np.where((img <= highThreshold) & (img >= lowThreshold))

        res[strong_i, strong_j] = strong
        res[weak_i, weak_j] = weak

        return (res)

    def hysteresis(self, img):

        M, N = img.shape
        weak = self.weak_pixel
        strong = self.strong_pixel

        for i in range(1, M-1):
            for j in range(1, N-1):
                if (img[i,j] == weak):
                    try:
                        if ((img[i+1, j-1] == strong) or (img[i+1, j] == strong) or (img[i+1, j+1] == strong)
                            or (img[i, j-1] == strong) or (img[i, j+1] == strong)
                            or (img[i-1, j-1] == strong) or (img[i-1, j] == strong) or (img[i-1, j+1] == strong)):
                            img[i, j] = strong
                        else:
                            img[i, j] = 0
                    except IndexError as e:
                        pass

        return img
    
    def detect(self, pic_names, save_path, num):
        imgs_final = []
        for i, img in enumerate(self.imgs):    
            if i == 0:
                print("Processing started ... ")
            if i < num:
                self.img_smoothed = convolve(img, self.gaussian_kernel(self.kernel_size, self.sigma))
                self.gradientMat, self.thetaMat = self.sobel_filters(self.img_smoothed)
                self.nonMaxImg = self.non_max_suppression(self.gradientMat, self.thetaMat)
                self.thresholdImg = self.threshold(self.nonMaxImg)
                img_final = self.hysteresis(self.thresholdImg)
                self.imgs_final.append(img_final)
                if (i+1)%50 == 0:
                    print(f"{i+1} pics out of {len(self.imgs)}")
                    save_sketches(self.imgs_final, pic_names, save_path)
            else: break
        print(f"{i+1} pics out of {len(self.imgs)}")
        save_sketches(self.imgs_final, pic_names, save_path)
        print("Done!")
        return self.imgs_final


/var/folders/rx/ny0wjdtj5hl5js4991nw48jr0000gn/T/ipykernel_14287/2110912475.py:8: DeprecationWarning: Please use `convolve` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import convolve


## Load Images from small set and detect

In [2]:
%matplotlib inline

In [3]:
save_path = "../raw_data/imgs_people_canny"

In [4]:
def save_sketches(imgs_final, pic_names, save_path):
    print("Converting & Saving ... ")
    
    if os.path.isdir(save_path) == False:
        os.mkdir(save_path)
    
    #print(pic_names[:10])
    for i, image in enumerate(imgs_final):
        image = np.invert(image)
        image = Image.fromarray(np.uint8((image)))
        
        name = pic_names[i]
        image = image.save(f"{save_path}/{name}")

In [6]:
def make_sketches(dir_name, save_path, num):
    imgs, pic_names = load_data(dir_name,stop=num)
    detector = cannyEdgeDetector(imgs, sigma=1.4, 
                                 kernel_size=5, 
                                 lowthreshold=0.01, 
                                 highthreshold=0.17, 
                                 weak_pixel=100)
    
    print("Detecting Edges ...")
    imgs_final = detector.detect(pic_names, save_path, num)
    
    
    print(len(imgs_final))
    
    return imgs_final, pic_names

# detecting and saving of sketches

In [18]:
images, pic_names = make_sketches('../raw_data/imgs_people/', save_path, num=17)

Detecting Edges ...
Processing started ... 


/var/folders/rx/ny0wjdtj5hl5js4991nw48jr0000gn/T/ipykernel_14287/1908222727.py:84: DeprecationWarning: Please use `convolve` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  Ix = ndimage.filters.convolve(img, Kx)
/var/folders/rx/ny0wjdtj5hl5js4991nw48jr0000gn/T/ipykernel_14287/1908222727.py:85: DeprecationWarning: Please use `convolve` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  Iy = ndimage.filters.convolve(img, Ky)


RuntimeError: filter weights array has incorrect shape.